In [1]:
# initialization - step 1

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [2]:
# initialization - step 2

!mkdir -p drive
!google-drive-ocamlfuse drive

In [22]:
from sklearn.cluster import KMeans
import numpy as np

# [, 128] 

#data = np.random.rand(1000, 128)
#filepath = 'drive/GGLandmark/clusters.npy'

def findcluster(data):
  clusters = 20

  kmeans = KMeans(
      n_clusters = clusters,
      init = 'k-means++',
      n_init = 10,
      max_iter = 200,   
  )

  kmeans.fit(data)
  np.save(filepath, kmeans.cluster_centers_)
  return kmeans.cluster_centers_
  
#clusters = findcluster(data)
#np.load(filepath)

array([[0.46945684, 0.51493724, 0.58507394, ..., 0.60880702, 0.54442408,
        0.57553245],
       [0.4425623 , 0.39054326, 0.46852217, ..., 0.40296884, 0.42114415,
        0.56537502],
       [0.59785428, 0.48747072, 0.58043531, ..., 0.59189266, 0.46623753,
        0.48013383],
       ...,
       [0.62551959, 0.52135719, 0.31512249, ..., 0.48553705, 0.57819229,
        0.45096453],
       [0.63886863, 0.46767919, 0.59799726, ..., 0.64243517, 0.31986965,
        0.4937856 ],
       [0.33689382, 0.49187382, 0.38943136, ..., 0.50230376, 0.4668944 ,
        0.40582131]])

In [25]:
def bagofwords(code, clusters, n_clusters):
  clusters = np.load(filepath)
  
  # new kmeans with existing clusters
  kmeans = KMeans(n_clusters = n_clusters)
  kmeans.cluster_centers_ = clusters
  
  # predict the new image
  labels = kmeans.predict(code)
  bow = np.zeros(n_clusters)
  for i in labels:
    bow[i] += 1.0
  
  return bow

code = np.random.rand(400, 128)
print(bagofwords(code, clusters, 20))

[13. 21. 18. 11. 32. 18. 11. 33. 25. 16.  5.  9. 17. 26. 27. 27. 28. 20.
 34.  9.]
